In [1]:
import numpy as np
import pandas as pd
import time
import datetime
from pathlib import Path
import os
import gc
import random
import nltk
import itertools
from nltk.corpus import stopwords
from typing import Literal, Callable
import re
from torch.utils.data import Dataset
from functools import partial

import torch
from torch import Tensor
import torch.nn as nn
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler,
    random_split,
)
from torch.optim import Adam
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import (
    BertForSequenceClassification,
    AdamW,
    BertConfig,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)

from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassPrecision,
    MulticlassRecall,
    MulticlassF1Score,
    MulticlassConfusionMatrix,
)

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar

In [2]:
Path_t = str | bytes | os.PathLike

In [3]:
RANDOM_SEED = 123
pl.seed_everything(RANDOM_SEED)

Global seed set to 123


123

In [4]:
device = torch.device("cuda")
torch.set_float32_matmul_precision("high")

In [15]:
NLTK_DIR = Path("./nltk")
MODEL_DIR = Path("./models")

In [5]:
nltk.download("stopwords", download_dir=NLTK_DIR)

[nltk_data] Downloading package stopwords to /home/tomek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS


class TextClassificationDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data_path: Path_t,
        tokenizer: BertTokenizer,
        batch_size: int,
        val_fraction: float = 0.0,
        truncation_strategy: Literal["truncate"] = "truncate",
        max_seq_len: int = 512,
        clean_fn: Callable[[str], str] = lambda x: x,
        n_first: int | None = None  # just for testing purposes
    ):
        super(TextClassificationDataModule, self).__init__()
        assert 0 <= val_fraction <= 1

        self._data_path = data_path
        self._clean_fn = clean_fn
        self._tokenizer = tokenizer
        self._encode_fn = partial(
            tokenizer.encode_plus,
            add_special_tokens=True,
            max_length=max_seq_len,
            truncation=True if truncation_strategy == "truncate" else False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
        )
        self.val_fraction = val_fraction
        self.batch_size = batch_size
        
        self._n_first = n_first

        self._encoded: list[dict[str, Tensor]]
        self._train: Dataset
        self._val: Dataset
        self._test: Dataset

    def setup(self, stage: Literal["fit", "test"]) -> None:
        if stage == "fit":
            df_raw = pd.read_csv(self._data_path)
            ratings = df_raw["rating"].values[:self._n_first]
            dataset = TensorDataset(
                *(
                    torch.cat([item[k] for item in self._encoded], dim=0)
                    for k in ("input_ids", "attention_mask")
                ),
                torch.tensor(ratings),
            )
            val_size = int(len(dataset) * self.val_fraction)
            train_size = len(dataset) - val_size

            self._train, self._val = torch.utils.data.random_split(
                dataset, [train_size, val_size]
            )

        if stage == "test":
            self._test = TensorDataset(
                *(
                    torch.cat([item[k] for item in self._encoded], dim=0)
                    for k in ("input_ids", "attention_mask")
                )
            )

    def prepare_data(self) -> None:
        df_raw = pd.read_csv(self._data_path)
        reviews = df_raw["review"].apply(self._clean_fn).values[:self._n_first]
        self._encoded = [self._encode_fn(review) for review in reviews]

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self._train, batch_size=self.batch_size, shuffle=True, pin_memory=True
        )

    def val_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self._val, batch_size=self.batch_size, shuffle=False, pin_memory=True
        )

    def test_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self._test, batch_size=self.batch_size, shuffle=False, pin_memory=True
        )

In [7]:
from typing import Any


class BertClassifier(pl.LightningModule):
    def __init__(
        self,
        model: BertForSequenceClassification,
        lr: float,
        weight_decay: float = 0.0,
        warmup_steps: int = 0,
        freeze_encoder: bool = False,
    ):
        super(BertClassifier, self).__init__()
        num_classes = model.num_labels
        self.num_classes = num_classes

        self.model = model
        self._lr = lr
        self._weight_decay = weight_decay
        self._warmup_steps = warmup_steps
        self._freeze_encoder = freeze_encoder

        if freeze_encoder:
            for param in self.model.base_model.parameters():
                param.requires_grad = False

        self.save_hyperparameters(ignore=["model"])

        self.flat_metrics = {
            "accuracy": MulticlassAccuracy(num_classes=num_classes, average="micro").to(device),
            "precision": MulticlassPrecision(num_classes=num_classes, average="micro").to(device),
            "recall": MulticlassRecall(num_classes=num_classes, average="micro").to(device),
            "f1": MulticlassF1Score(num_classes=num_classes, average="micro").to(device),
        }
        self.class_metrics = {
            "accuracy": MulticlassAccuracy(num_classes=num_classes, average="none").to(device),
            "precision": MulticlassPrecision(num_classes=num_classes, average="none").to(device),
            "recall": MulticlassRecall(num_classes=num_classes, average="none").to(device),
            "f1": MulticlassF1Score(num_classes=num_classes, average="none").to(device),
        }
        self.confusion_matrix = MulticlassConfusionMatrix(
            num_classes=num_classes, normalize="pred"
        ).to(device)

    def configure_optimizers(self) -> Any:
        self.trainer.fit_loop.setup_data()
        total_devices = getattr(self.hparams, "n_gpus", 1) * getattr(self.hparams, "n_nodes", 1)
        train_batches = len(self.trainer.train_dataloader) // total_devices
        train_steps = (
            self.trainer.max_epochs * train_batches
        ) // self.trainer.accumulate_grad_batches
        # will not work with AdamW (plus it is deprecated anyway)
        # https://discuss.huggingface.co/t/runtimeerror-element-0-of-tensors-does-not-require-grad-and-does-not-have-a-grad-fn/47965/2
        optimizer = Adam(self.model.parameters(), lr=self._lr, eps=1e-8)
        lr_scheduler = get_linear_schedule_with_warmup(
            optimizer=optimizer,
            num_warmup_steps=self._warmup_steps,
            num_training_steps=train_steps,
        )
        return [optimizer], [lr_scheduler]

    def training_step(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> dict[str, Tensor]:
        input_ids, input_mask, labels = batch
        output = self.model(
            input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels
        )
        loss = output.loss

        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> dict[str, Tensor]:
        with torch.no_grad():
            input_ids, input_mask, labels = batch
            output = self.model(
                input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels
            )
            loss = output.loss

            preds = output.logits
            self.log("val_loss", loss, prog_bar=True)
            self.log_dict(
                {name: metric(preds, labels) for name, metric in self.flat_metrics.items()},
                prog_bar=True,
            )
            self.log_dict(
                {
                    f"{name}_{c}": v
                    for name, metric_values in {
                        _name: metric(preds, labels)
                        for _name, metric in self.class_metrics.items()
                    }.items()
                    for c, v in enumerate(metric_values)
                },
                prog_bar=True,
            )
            self.confusion_matrix(preds, labels)

    def on_validation_epoch_end(self) -> None:
        with torch.no_grad():
            confusion_matrix = self.confusion_matrix.compute()

            df_cm = pd.DataFrame(
                confusion_matrix.cpu().numpy(), index=range(5), columns=range(5)
            )
            fig, ax = plt.subplots(figsize=(10, 7))
            sns.heatmap(
                df_cm,
                ax=ax,
                annot=True,
                cmap="Greens",
                vmin=0,
                vmax=1,
                annot_kws={"size": 6},
                fmt=".3f",
                
            )
            ax.set_xlabel("True class")
            ax.set_ylabel("Predicted class")
            self.logger.experiment.add_figure("Confusion matrix", fig, self.current_epoch)

    def predict_step(self, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> Any:
        with torch.no_grad():
            input_ids, input_mask, labels = batch
            return self.model(
                input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels
            )

In [16]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True, cache_dir=MODEL_DIR)

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
nltk.download('stopwords', download_dir="./nltk")
sw = stopwords.words('english')

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text)
    return text

[nltk_data] Downloading package stopwords to ./nltk...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
dm = TextClassificationDataModule(
    data_path=Path("data/train_data.csv"),
    tokenizer=tokenizer,
    val_fraction=0.2,
    batch_size=32,
    clean_fn=clean_text
)
dm.prepare_data()
dm.setup("fit")

In [11]:
model = BertClassifier(
    model=BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=5,
        output_attentions=False,
        output_hidden_states=False,
        cache_dir=MODEL_DIR
    ),
    lr=2e-5,
    weight_decay=0,
    warmup_steps=0,
    freeze_encoder=False,
)
model

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertClassifier(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768,

In [12]:
trainer = pl.Trainer(
    max_epochs=4,
    gradient_clip_val=1,
    callbacks=[RichProgressBar()],
    precision="bf16",
    val_check_interval=0.2
)
trainer.fit(model, datamodule=dm)


/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/lightning_fabric/connector.py:555: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/tomek/ssne/proj6/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                          ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model            │ BertForSequenceClassification │  109 M │
│ 1 │ confusion_matrix │ MulticlassConfusionMatrix     │      0 │
└───┴──────────────────┴───────────────────────────────┴────────┘

Trainable params: 109 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 109 M                                                                                                
Total estimated model params size (MB): 437

Output()

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connecto
r.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on 
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

...

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: 
10 NaN values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)  # noqa: B028

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: 
5 NaN values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)  # noqa: B028

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: 
5 NaN values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)  # noqa: B028

/home/tomek/miniconda3/envs/ssne_p3/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: 
10 NaN values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)  # noqa: B028

`Trainer.fit` stopped: `max_epochs=4` reached.
